In [197]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import chromadb

In [198]:
reliable_articles = pd.read_csv("Data/Politifact_Data/CSV/politifact_articles.csv")
reliable_articles = reliable_articles.drop(columns='Unnamed: 0')
reliable_articles.rename(columns={'Statement': 'Title'}, inplace=True)
reliable_articles = reliable_articles.dropna()

In [229]:
pf_true_statements = pd.read_csv("Data/Politifact_Data/CSV/true_df.csv")
pf_true_statements = pf_true_statements.drop(columns='Unnamed: 0')
pf_true_statements = pf_true_statements.dropna()
pf_true_statements

,Claimer,Statement,Truth_value,Tldr_text_statements,Text
0,Melissa Agard,"“Since 1981, the state Senate has only rejecte...",true,['The state Senate voted to fire eight of the ...,The Republican-controlled state Senate on Oct....
1,Instagram posts,“Lego donates model MRI kits to hospitals to h...,true,['A Lego employee and physicians at a Danish h...,"Undergoing a medical procedure, such as an MRI..."
2,Brian Krassenstein,It has been U.S. policy “for at least 79 years...,true,['Many airlines have canceled flights from Isr...,"Editor’s note, Oct. 18: After publication, we ..."
3,Mike Oliverio,"“A few years back,” Mitchell Stadium in Bluefi...",true,"['In a USA Today contest in 2019, Bluefield’s ...",As the 2023 high school football season got un...
4,Instagram posts,"""Beyonce paid $100K for Maryland Metro to run ...",true,"['Beyoncé’s Renaissance World Tour paid $100,0...","Washington, D.C.-area concertgoers flocking to..."
...,...,...,...,...,...
2575,Hillary Clinton,"Health insurance premiums ""have almost doubled...",true,[],"In a speech unveiling her health care plan, Se..."
2576,John Edwards,"Poor people go to a ""payday lender...and they ...",true,[],"At a campaign stop in Americus, Georgia, John ..."
2577,Fred Thompson,"""I've cast a couple of 99-1 votes"" and been th...",true,[],"\nAlthough not every senator voted every time,..."
2578,Hillary Clinton,\u201CThirty-four percent of Hispanics don\u20...,true,[],"\nAt Sunday night\u2019s debate in Miami, Demo..."


In [230]:
def tokenize_into_sentences(text):
    return sent_tokenize(text)

In [231]:
def tokenize_into_chunks(text, min_words=75):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        if len(current_chunk) + len(words) < min_words:
            current_chunk.extend(words)
        else:
            if any(sentence.endswith(p) for p in ['.', '!', '?', '¡', '¿']):
                chunks.append(' '.join(current_chunk))
                current_chunk = words
            else:
                current_chunk.extend(words)

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

In [232]:
#Politifact True Statement Text Chunking
pf_true_statements['chunks'] = pf_true_statements['Text'].apply(tokenize_into_chunks)

max_chunks = pf_true_statements['chunks'].apply(len).max()

for i in range(1, max_chunks + 1):
    pf_true_statements[f'chunk_{i}'] = pf_true_statements['chunks'].apply(lambda x: x[i - 1] if len(x) >= i else None)

pf_true_statements = pf_true_statements.drop(columns=['chunks'])

In [201]:
#Politifact Articles Chunking
reliable_articles['chunks'] = reliable_articles['Text'].apply(tokenize_into_chunks)

max_chunks = reliable_articles['chunks'].apply(len).max()

for i in range(1, max_chunks + 1):
    reliable_articles[f'chunk_{i}'] = reliable_articles['chunks'].apply(lambda x: x[i - 1] if len(x) >= i else None)

reliable_articles = reliable_articles.drop(columns=['chunks'])

/var/folders/pb/7hrkp8tj05gbr513hzd6bv000000gp/T/ipykernel_82078/3380665291.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  reliable_articles[f'chunk_{i}'] = reliable_articles['chunks'].apply(lambda x: x[i - 1] if len(x) >= i else None)
/var/folders/pb/7hrkp8tj05gbr513hzd6bv000000gp/T/ipykernel_82078/3380665291.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  reliable_articles[f'chunk_{i}'] = reliable_articles['chunks'].apply(lambda x: x[i - 1] if len(x) >= i else None)
/var/folders/pb/7hrkp8tj05gbr513hzd6bv000000gp/T/ipy

In [269]:
reliable_articles

,Title,Tldr_text_statements,Text,chunk_1,chunk_2,chunk_3,chunk_4,chunk_5,chunk_6,chunk_7,...,chunk_164,chunk_165,chunk_166,chunk_167,chunk_168,chunk_169,chunk_170,chunk_171,chunk_172,chunk_173
0,Did Mike Johnson work for a hate group? SPLC a...,[],"Since U.S. Rep. Mike Johnson, R-La., won the O...","Since U.S. Rep. Mike Johnson , R-La. , won the...",Johnson previously worked for the Alliance Def...,`` The man who was just elected speaker of the...,The law center and the alliance have sparred f...,"For example , the Center for Immigration Studi...",The Alliance Defending Freedom has racked up 1...,"`` Extremists like ADF , who use religious rhe...",...,None,None,None,None,None,None,None,None,None,None
1,Ask PolitiFact: How many people on the terrori...,['The FBI’s terrorist watchlist is a federal d...,Some Republican lawmakers are flagging Hamas’ ...,Some Republican lawmakers are flagging Hamas ’...,"In September alone , 18 illegal immigrants on ...","`` We just caught 18 people , just last month ...","But not everyone on the list is a terrorist , ...",`` People aren ’ t crossing the border to cond...,A Blackburn spokesperson pointed us to a Fox N...,"The terrorist watchlist , run by the FBI ’ s T...",...,None,None,None,None,None,None,None,None,None,None
2,Mike Johnson sought to overturn 2020 election....,"['In December 2020, U.S. Rep. Mike Johnson, R-...",After House Republicans unanimously voted for ...,After House Republicans unanimously voted for ...,Johnson ’ s efforts went beyond tweets and vot...,"Rep. Pete Aguilar , D-Calif. , said on the Hou...",Republicans booed ABC News reporter Rachel Sco...,Johnson was not among those charged in the fed...,"However , Johnson played a key role in the eff...",Johnson aligned himself with Trump ’ s respons...,...,None,None,None,None,None,None,None,None,None,None
3,Did Donald Trump make these 27 campaign promis...,[],A liberal group says former President Donald T...,A liberal group says former President Donald T...,The article was written by MeidasTouch ’ s edi...,Many of Trump ’ s promises would face practica...,Former President Donald Trump smiles before hi...,`` Will arrest all homeless people across the ...,But he also said that relocation and rehabilit...,This is essentially what Trump said . In a vid...,...,None,None,None,None,None,None,None,None,None,None
4,Guerra de Israel-Hamas: Cómo se esparcieron re...,['Después de que el grupo militante palestino ...,Nota del Editor: Este artículo contiene refere...,Nota del Editor : Este artículo contiene refer...,Pero en particular una perturbadora declaració...,"Desde el ataque , la declaración ha sido ampli...",La devastación de la guerra se ha intensificad...,( Un informe de inteligencia estadounidense de...,La violencia confirmada ya es lo suficientemen...,`` Porque es una declaración impactante … ha o...,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Congress blocked Obama's call for new gun laws...,[],It was an issue over which President Barack Ob...,It was an issue over which President Barack Ob...,His eight years in office included many more h...,Gunmen with semi-automatic rifles or handguns ...,Obama ’ s first-term record was more focused o...,"Overall , adding any restrictions was a non-st...","With little interest from Congress , renewing ...",`` The gun control-gun rights debate is alread...,...,None,None,None,None,None,None,None,None,None,None
2996,"Hate crimes, civil rights become law enforceme...",[],"President Barack Obama took office vowing to ""...",President Barack Obama took office vowing to `...,He fulfilled a pledge to support the Matthew S...,"Before the law was passed , there were 45 stat...",`` There were a lot of cases that went forward...,The Federal Bureau of Investigation also prepa...,It ’ s important to remember a couple things a...,Another is that any increase in reported cases...,...,None,None,None,None,None

In [202]:
chroma_client = chromadb.Client()

In [203]:
pf_vector_db = chroma_client.create_collection(name="pf_vector_db")

In [222]:
#Politifact Text breakdown
chunks_list = []
titles_list = []
ids_list = []
start_id = 60579

for index, row in pf_true_statements.iterrows():
    title = row['Statement']
    for col in pf_true_statements.columns:
        if col.startswith('chunk_'):
            chunk = row[col]
            if chunk is not None:
                chunks_list.append(chunk)
                titles_list.append({"Title": title})
                ids_list.append(f"id{start_id}")
                start_id += 1

In [261]:
#Politifact Claimer and Statement break down
statements_list = []
claimers_list = []
ids_list = []
start_id = 94049

for index, row in pf_true_statements.iterrows():
    claimer = row['Claimer']
    statement = row['Statement']
    
    claimers_list.append({"Claimer": claimer})
    statements_list.append(statement)
    ids_list.append(f"id{start_id}")
    start_id += 1

In [262]:
len(ids_list)

2572

In [263]:
start_size = 0
batch_size_increment = 5000
batch_size = 5000
for i in range(((len(chunks_list)//batch_size)+1)):
    pf_vector_db.add(
        documents=statements_list[start_size:batch_size],
        metadatas=claimers_list[start_size:batch_size],
        ids=ids_list[start_size:batch_size])
    start_size = start_size + batch_size_increment
    batch_size = batch_size + batch_size_increment
    print(start_size)

5000


ValueError: Expected IDs to be a non-empty list, got []

In [264]:
pf_vector_db.count()

96620

In [271]:
pf_vector_db.query(
    query_texts=['Did Beyonce pay 100K for Maryland Metro to run later for a concert of hers?'],
    n_results=1,
)

{'ids': [['id94053']],
 'distances': [[0.27881860733032227]],
 'metadatas': [[{'Claimer': 'Instagram posts'}]],
 'embeddings': None,
 'documents': [['"Beyonce paid $100K for Maryland Metro to run later following weather delays for her concert.”']],
 'uris': None,
 'data': None}